# My New Data Playground

Small data is still valuable data, no matter what the job listings say. In my work with Table Wisdom I've found myself doing highly repetitve, inefficient tasks in order to develop a sense of how we can recruit efficiently. It was old after the third time, and two weeks in it was absolutely mind numbing. 

What better time to bring out my two favorite toys, Python and Pandas, to make life a little more beautiful. In what follows I share the processes I've developed to automate exporting general data concerning our volunteer postings, and then cleaning it and putting it together in a useful file format. At the end of it all connect to a Google Sheet I've shared through the Drive API and upload my workbook.

The whole process took me about a day, with a lot of new concepts in it for me. All in all I'm super happy with the result. I've actually been laughing out loud when things work. Pure joy.

In [1]:
import datetime as dt
import os

# I'm using gspread and oauth2 to connect to Google's API. Openpyxl is helping me add new sheets to an existing
# workbook without overwriting my original data.
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from openpyxl import load_workbook
import pandas as pd

In [20]:
base = "c:\\Path\\To\\Spreadsheets\\"

### It took me ages to figure out what was going wrong with this first function

Ok, maybe it was 30 minutes. But it felt like ages. In a separate script I run a selenium webdriver to log in to my account on VolunteerMatch and export our opportunity reports. I wired up a Firefox profile to circumnavigate the "Save as" pop-up window, and an '.xls' file came through on the other end. 

Then I tried to run pd.read_excel and I got a Beginning of File error. Apparently a .xls file isn't meant to begin with "Title\tCon...". I'm sure most of you would look at that and realize the issue straight away. This was my first encounter of the BOF kind, so I hit Stack Overflow. A few solutions suggested trying pd.read_html, which I tried. The response? "No table found." Well of course not, there's no HTML table ta...

It was a tab delimited file: I needed pd.read_table. 

In [3]:
def format_data():
    # Our data has a '.xls' extension, but it's really a tab delimited file
    df = pd.read_table(base + "report.xls")

    # Filtering out unneccessary columns
    headers =['Title', 'Connections', 'Views', 'Date', 'City', 'State', 'ZIP Code']
    vm_data = df[headers]

    # Want a column showing rate of response (RR)
    connections = vm_data.loc[:,'Connections']
    views = vm_data.loc[:,'Views']
    vm_data['RR'] = connections/views

    # We also want to show values based on response rate, highest first
    zip_order = vm_data.sort_values('ZIP Code', ascending=True).loc[:,
                    ['ZIP Code', 'RR', 'Connections', 'Views', 'Title',]]

    zip_order = zip_order.rename(columns={'ZIP Code': 'Zip Code'})
    return zip_order

This one is pretty much identical to the function above, but I needed to be able to pair up zip codes, cities, and states while I was throwing together a quick function to get list some data I was missing.

In [4]:
def format_data_full():
    # Our data has a '.xls' extension, but it's really a tab delimited file
    df = pd.read_table(base + "report.xls")

    # Filtering out unneccessary columns
    headers =['Title', 'Connections', 'Views', 'Date', 'City', 'State', 'ZIP Code']
    vm_data = df[headers]

    # Want a column showing rate of response (RR)
    connections = vm_data.loc[:,'Connections']
    views = vm_data.loc[:,'Views']
    vm_data['RR'] = connections/views

    # We also want to show values based on response rate, highest first
    zip_order = vm_data.sort_values('ZIP Code', ascending=True).loc[:,
                    ['ZIP Code', 'RR', 'Connections', 'Views', 'Title', 'City', 'State']]

    zip_order = zip_order.rename(columns={'ZIP Code': 'Zip Code'})
    return zip_order

## Just wanted to show off my web scraping script.

I won't run it here, but this is my script to automatically download our Opportunity report. 

In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile
from login import login

url = "https://www.volunteermatch.org/track/need_to_know_basis"

def get_vm_connections():
    # Disabling the "save" dialog box via custom profile settings
    profile = FirefoxProfile()
    profile.set_preference("browser.download.panel.shown", False)
    profile.set_preference("browser.helperApps.neverAsk.openFile","text/xls,application/vnd.ms-excel")
    profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "text/xls,application/vnd.ms-excel")
    profile.set_preference("browser.download.folderList", 2);
    profile.set_preference("browser.download.dir", "c:\\Users\\wolfa\\Documents\\VolunteerMatch\\")

    driver = webdriver.Firefox(firefox_profile=profile)
    driver.get(url)

    # Autofilling username and password to log in
    username = driver.find_element_by_id("username")
    password = driver.find_element_by_id("password")

    username.send_keys(login['username'])
    password.send_keys(login['password'])

    driver.find_element_by_id("login_button").click()

    # This is the "export opp report" button
    driver.find_element_by_class_name("adminui_button").click()

    driver.close()

get_vm_connections()


### Learning about Google Drive's API was a lot of fun

Now that I know how to work with it I have a feeling I'll be doing a lot more remote work with my Drive files. I opted to replace all of the data in the workbook rather than trying to start a new sheet for each upload. I figure having the original files on my desktop is good enough, we won't need historical analysis all that often.

In [5]:
def upload_to_sheets(dataframe):
    # use creds to create a client to interact with Drive API
    scope = [
        'https://spreadsheets.google.com/feeds',
        'https://www.googleapis.com/auth/drive',
    ]
    
    creds = ServiceAccountCredentials.from_json_keyfile_name('secret.json', scope)
    client = gspread.authorize(creds)

    client.import_csv("abunchofrandomcharacters", dataframe.to_csv())


### I ran into some trouble adding new sheets to a workbook

Again, Stack Overflow to the rescue. The save_to_excel function is a super simple version of what I really wanted to do, and will overwrite the workbook every time. The append_vm_data function is what I was really after. Using the openpyxl engine makes sure my forumlae don't break when I write to the workbook, and I'm calling pandas ExcelWriter explicitly to get that beautiful new sheet functionality. 

In [6]:
def save_to_excel(df):
    today = dt.datetime.today().strftime("%m-%d-%Y")
    df.to_excel(base + 'vm-data1.xlsx',
                sheet_name='{}'.format(today))

In [7]:
def append_vm_data(df):
    # Want to make sure the sheet name reflects when I ran my script
    today = dt.datetime.today().strftime("%m-%d-%Y")
    
    book = load_workbook(base + 'vm-opp-data.xlsx')
    writer = pd.ExcelWriter(base + 'vm-opp-data.xlsx', engine='openpyxl')
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
    df.to_excel(writer, today)
    writer.save()

## Pandas really likes turning things into float type

Rather than write out all of my converters when building my dataframe, I elected to put them in a dictionary. Anything to stay pythonic.

In [8]:
type_def = {
    'Zip Code':int,
    'Population':int,
    'Age 50+':float,
    'Potential Reach':int,
    "Bachelor's Degree+":float,
    'Median HH Income':int,
    'Households w/ Retirement Income':float,
    'City':str,
    'State':str,
}

In [9]:
opp_data = format_data()
opp_data.tail()

c:\users\wolfa\.virtualenvs\tw\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


,Zip Code,RR,Connections,Views,Title
24,90209,0.029046,7,241,Do You Like To Chat? Have fun and Videochat wi...
0,94129,0.020000,3,150,Do You Like to Chat? Have Fun and Video-chat ...
2,97034,0.013793,2,145,Do You Like to Chat? Have Fun and Video-chat ...
7,97301,0.000000,0,56,Do You Like to Chat? Have Fun and Video-chat ...
1,98039,0.009615,2,208,Do You Like to Chat? Have Fun and Video-chat ...


In [10]:
demo_data = pd.read_excel(base + 'vm-opp-data.xlsx',sheetname='Template',converters=type_def)
demo_data.tail()

,Zip Code,Population,Age 50+,Potential Reach,Bachelor's Degree+,Median HH Income,Households w/ Retirement Income,City,State
31,94129,3639,0.10,363,0.86,163786,0.01,San Francisco,CA
32,97034,19290,0.47,9066,0.69,104571,0.19,Lake Oswego,OR
33,97301,54844,0.24,13162,0.15,36455,0.17,Salem,OR
34,98039,3120,0.42,1310,0.82,174063,0.11,Medina,WA
35,90209,22052,0.45,9923,0.61,145227,0.10,Beverly Hills,CA


## My original spreadsheet with income data was missing some zip codes

I threw together a few lists to quickly parse which zip codes I was missing. We've been using [Town Charts](towncharts.com) for most of our analysis, and thankfully their url structure is very easy to use. Rather than typing zip codes into their search box, I elected to generate the urls to the zip codes I needed with a clean for-loop and leisurely click through the list. This made me very happy.

Since I've finished building out my dataset at this point, I've added one errant zip code to demonstrate functionality. Pandas is doing its float thing, so I convert to int explicitly in case you want to click the link to make sure it works.

In [12]:
opp_zips = []
res_zips = []

for item in opp_data['Zip Code']:
    opp_zips.append(item)

for item in demo_data['Zip Code']:
    res_zips.append(item)

missing_zips =[]

for code in opp_zips:
    if code not in res_zips:
        missing_zips.append(code)

In [13]:
df = format_data_full()
states = df.loc[df['Zip Code'].isin(missing_zips), ['State', 'Zip Code']]
states.loc[0,['Zip Code']] = 63103
states.loc[0,['State']] = 'MO'
print(states)
abb_to_full = {
    'NY': 'New-York',
    'NC': 'North-Carolina',
    'FL': 'Florida',
    'TN': 'Tennessee',
    'IA': 'Iowa',
    'MN': 'Minnesota',
    'TX': 'Texas',
    'MO': 'Missouri',
    'CO': 'Colorado',
    'UT': 'Utah',
    'OR': 'Oregon',
    'CA': 'California',
    'OR': 'Oregon',
    'WA': 'Washington',
}

  State  Zip Code
0    MO   63103.0


c:\users\wolfa\.virtualenvs\tw\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [14]:
url_template = "http://www.towncharts.com/{}/Demographics/{}-Zipcode-{}-Demographics-data.html"

for index, row in states.iterrows():
    url = url_template.format(abb_to_full[row['State']], 
                              int(row['Zip Code']), 
                              row['State'])
    print(row['Zip Code'], url)

63103.0 http://www.towncharts.com/Missouri/Demographics/63103-Zipcode-MO-Demographics-data.html


## And the finishing touches

I wanted to have the connection and view data side-by-side with my demographic research. Thanks to pandas, it's as easy as a merge on Zip Code values. I love the Python community, so so much.

In [16]:
mergeDF = pd.merge(demo_data, opp_data, left_on=['Zip Code'], right_on=['Zip Code'])

In [18]:
mergeDF.head()

,Zip Code,Population,Age 50+,Potential Reach,Bachelor's Degree+,Median HH Income,Households w/ Retirement Income,City,State,RR,Connections,Views,Title
0,10023,60586,0.280,16964,0.81,111478,0.13,Manhattan,NY,0.047337,8,169,Share your experience! Mentor foreign born stu...
1,11803,28390,0.400,11356,0.60,126536,0.24,Nassau County,NY,0.000000,0,69,Share your experience! Mentor foreign born stu...
2,14618,22352,0.360,8046,0.68,89688,0.21,Rochester,NY,0.000000,0,92,Do You Like to Chat? Videochat and Mentor Our...
3,20744,51759,0.450,23291,0.33,88384,0.35,Fort Washington,MD,0.032967,3,91,Share your experience! Mentor foreign born stu...
4,22032,29956,0.365,10933,0.66,140253,0.31,Fairfax,VA,0.013333,1,75,Share your experience! Mentor foreign born stu...


Then I call my append function to place a timestamped worksheet in my local workbook, and upload the final dataframe to Google Sheets. The local version is prettier, but the pure dataframe translation gets the job done.

In [ ]:
append_vm_data(mergeDF)

In [ ]:
upload_to_sheets(mergeDF)

## Today was a good day

I'm feeling at one with ol-boy Ice Cube. I've simplified my workflow significantly and can focus on analysis and success rather than playing around with logins and manually formatting excel files. I'm rather pleased. Tomorrow I'll be digging in to what's working with our targeting and developing strategies we can use to improve our rate of response. Separately, I'm trying to ween myself from Town Charts and start collecting the ACS data directly from the Census website. This is a much taller order and will probably take a bit longer, but when i get it working -- Oh the fun we will have!

Big thanks to the Stack Overflow community and all of the other people that tripped over my errors first so I didn't have to figure everything out on my own. 

Cheers!

_Wolf_